## Add hexapod motions to the mount plots

Craig Lage  04-Jun-25

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lsst.daf.butler import Butler
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData
from lsst.summit.utils.simonyi.mountAnalysis import calculateMountErrors, plotMountErrors
from lsst.summit.utils.butlerUtils import getExpRecordFromDataId
from scipy.optimize import minimize

In [ ]:
client = makeEfdClient()
butler = Butler('/repo/embargo', collections=['LSSTCam/raw/all', 
                                            'LSSTCam/calib/unbounded', 'LSSTCam/runs/nightlyValidation'])

In [ ]:
expId = 2025082800505
dataId = {'exposure':expId, 'instrument':'LSSTCam'}
expRecord = getExpRecordFromDataId(butler, dataId)

In [ ]:
(mountErrors, mountData) = calculateMountErrors(expRecord, client)
print(mountErrors)
saveFilename = f"/home/c/cslage/u/MTMount/mount_plots/Mount_Plot_Hex_{expId}.png"
fig = plotMountErrors(mountData, mountErrors, saveFilename=saveFilename)

In [ ]:
mountData.azimuthData['actualPositionTimestamp'].values[27]

In [ ]:
aTimes = mountData.azimuthData['actualPositionTimestamp'].values
aVals = mountData.azimuthData['actualPosition'].values
eTimes = mountData.elevationData['actualPositionTimestamp'].values
eVals = mountData.elevationData['actualPosition'].values
dATs = []
dAVs = []
aSlopes = []
eSlopes = []
for i in range(len(aVals)):
    dAT = (aTimes[i] - aTimes[i-1])
    dET = (eTimes[i] - eTimes[i-1])
    dAV = (aVals[i] - aVals[i-1])
    dEV = (eVals[i] - eVals[i-1])
    aSlopes.append(dAV/dAT)
    eSlopes.append(dEV/dET)
    dATs.append(dAT)
    dAVs.append(dAV)
plt.plot(aSlopes, marker='x')
plt.plot(eSlopes, marker='x')
plt.xlim(20,30)

In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot(dATs[20:30], marker='x', color='blue')
ax1 = ax.twinx()
ax1.plot(dAVs[20:30], marker='+', color='red')


In [ ]:
plt.plot(aTimes[20:30], aVals[20:30], marker='x')

In [ ]:
expId = 2025082800505
dataId = {'exposure':expId, 'instrument':'LSSTCam'}
expRecord = getExpRecordFromDataId(butler, dataId)
mountData.azimuthData['actualPosition']
(mountErrors, mountData) = calculateMountErrors(expRecord, client)
az = mountData.azimuthData['actualPosition'].values
el = mountData.elevationData['actualPosition'].values
print(abs(az[0] - az[-1]), abs(el[0] - el[-1]))

In [ ]:
from lsst.summit.utils.tmaUtils import TMAEventMaker
eventMaker = TMAEventMaker()
events = eventMaker.getEvents(20250828)
tracks = [event for event in events if event.type.name=='TRACKING']
tracks[7]

In [ ]:
fig = plt.figure(figsize = (10,10))
plt.subplots_adjust(wspace=0.5)
plt.suptitle(f"TMA tracking errors {expId}", fontsize=18)
actVals = mountData.azimuthData['actualPosition'].values
times = mountData.azimuthData['actualPositionTimestamp'].values
times -= times[0]
fit = np.polyfit(times, actVals, 1)
fitVals = np.polyval(fit, times)
plt.subplot(2,2,1)
plt.plot(times, (actVals - fitVals)*3600)
plt.ylim(-0.1, 0.1)
plt.title("Azimuth")
plt.ylabel("actualPosition deviation from linear fit (arcsec)")
plt.xlabel("Time (seconds)")
actVals = mountData.elevationData['actualPosition'].values
times = mountData.elevationData['actualPositionTimestamp'].values
times -= times[0]
fit = np.polyfit(times, actVals, 1)
fitVals = np.polyval(fit, times)
plt.subplot(2,2,2)
plt.plot(times, (actVals - fitVals)*3600)
plt.ylim(-0.1, 0.1)
plt.title("Elevation")
plt.ylabel("actualPosition deviation from linear fit (arcsec)")
plt.xlabel("Time (seconds)")
actVals = mountData.azimuthData['actualPosition'].values
times = mountData.azimuthData['actualPositionTimestamp'].values
times -= times[0]
fit = np.polyfit(times, actVals, 2)
fitVals = np.polyval(fit, times)
plt.subplot(2,2,3)
plt.plot(times, (actVals - fitVals)*3600)
plt.ylim(-0.1, 0.1)
plt.title("Azimuth")
plt.ylabel("actualPosition deviation from quadratic fit (arcsec)")
plt.xlabel("Time (seconds)")
actVals = mountData.elevationData['actualPosition'].values
times = mountData.elevationData['actualPositionTimestamp'].values
fit = np.polyfit(times, actVals, 2)
fitVals = np.polyval(fit, times)
plt.subplot(2,2,4)
plt.plot(times, (actVals - fitVals)*3600)
plt.ylim(-0.1, 0.1)
plt.title("Elevation")
plt.ylabel("actualPosition deviation from quadratic fit (arcsec)")
plt.xlabel("Time (seconds)")

plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/TMA_ActualPosition_{expId}.png")

In [ ]:
fig = plt.figure(figsize = (10,10))
plt.subplots_adjust(wspace=0.5)
plt.suptitle(f"TMA tracking errors {expId}", fontsize=18)
demVals = mountData.azimuthData['demandPosition'].values
times = mountData.azimuthData['demandPositionTimestamp'].values
times -= times[0]
fit = np.polyfit(times, demVals, 1)
fitVals = np.polyval(fit, times)
plt.subplot(2,2,1)
plt.plot(times, (demVals - fitVals)*3600)
plt.ylim(-0.1, 0.1)
plt.title("Azimuth")
plt.ylabel("demandPosition deviation from linear fit (arcsec)")
plt.xlabel("Time (seconds)")
demVals = mountData.elevationData['demandPosition'].values
times = mountData.elevationData['demandPositionTimestamp'].values
times -= times[0]
fit = np.polyfit(times, demVals, 1)
fitVals = np.polyval(fit, times)
plt.subplot(2,2,2)
plt.plot(times, (demVals - fitVals)*3600)
plt.ylim(-0.1, 0.1)
plt.title("Elevation")
plt.ylabel("demandPosition deviation from linear fit (arcsec)")
plt.xlabel("Time (seconds)")
demVals = mountData.azimuthData['demandPosition'].values
times = mountData.azimuthData['demandPositionTimestamp'].values
times -= times[0]
fit = np.polyfit(times, demVals, 2)
fitVals = np.polyval(fit, times)
plt.subplot(2,2,3)
plt.plot(times, (demVals - fitVals)*3600)
plt.ylim(-0.1, 0.1)
plt.title("Azimuth")
plt.ylabel("demandPosition deviation from quadratic fit (arcsec)")
plt.xlabel("Time (seconds)")
demVals = mountData.elevationData['demandPosition'].values
times = mountData.elevationData['demandPositionTimestamp'].values
fit = np.polyfit(times, demVals, 2)
fitVals = np.polyval(fit, times)
plt.subplot(2,2,4)
plt.plot(times, (demVals - fitVals)*3600)
plt.ylim(-0.1, 0.1)
plt.title("Elevation")
plt.ylabel("demandPosition deviation from quadratic fit (arcsec)")
plt.xlabel("Time (seconds)")

plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/TMA_DemandPosition_{expId}.png")

In [ ]:
mtPos = getEfdData(client, 'lsst.sal.MTMount.azimuth',
                 expRecord=expRecord)

fig = plt.figure(figsize = (10,10))
plt.subplots_adjust(wspace=0.5)
plt.suptitle(f"TMA tracking errors {expId}", fontsize=18)
azVals = mtPos['demandPosition'].values
azTimes = mtPos['demandPositionTimestamp'].values
#azTimes = mtPos['timestamp'].values
azTimes -= azTimes[0]
fit = np.polyfit(azTimes, azVals, 1)
fitVals = np.polyval(fit, azTimes)
azErr = (azVals - fitVals)*3600
plt.subplot(2,2,1)
plt.plot(azTimes[100:110], azErr[100:110])
azVals = mtPos['actualPosition'].values
azTimes = mtPos['actualPositionTimestamp'].values
#azTimes = mtPos['timestamp'].values
azTimes -= azTimes[0]
fit = np.polyfit(azTimes, azVals, 1)
fitVals = np.polyval(fit, azTimes)
azErr = (azVals - fitVals)*3600
plt.plot(azTimes[100:110], azErr[100:110])
#plt.ylim(-0.1, 0.1)
plt.xlim(7.7, 8.5)
plt.title("Azimuth")
plt.ylabel("demandPosition deviation from linear fit (arcsec)")
plt.xlabel("Time (seconds)")
"""
demVals = mountData.elevationData['demandPosition'].values
times = mountData.elevationData['demandPositionTimestamp'].values
times -= times[0]
fit = np.polyfit(times, demVals, 1)
fitVals = np.polyval(fit, times)
plt.subplot(2,2,2)
plt.plot(times, (demVals - fitVals)*3600)
plt.ylim(-0.1, 0.1)
plt.title("Elevation")
plt.ylabel("demandPosition deviation from linear fit (arcsec)")
plt.xlabel("Time (seconds)")
demVals = mountData.azimuthData['demandPosition'].values
times = mountData.azimuthData['demandPositionTimestamp'].values
times -= times[0]
fit = np.polyfit(times, demVals, 2)
fitVals = np.polyval(fit, times)
plt.subplot(2,2,3)
plt.plot(times, (demVals - fitVals)*3600)
plt.ylim(-0.1, 0.1)
plt.title("Azimuth")
plt.ylabel("demandPosition deviation from quadratic fit (arcsec)")
plt.xlabel("Time (seconds)")
demVals = mountData.elevationData['demandPosition'].values
times = mountData.elevationData['demandPositionTimestamp'].values
fit = np.polyfit(times, demVals, 2)
fitVals = np.polyval(fit, times)
plt.subplot(2,2,4)
plt.plot(times, (demVals - fitVals)*3600)
plt.ylim(-0.1, 0.1)
plt.title("Elevation")
plt.ylabel("demandPosition deviation from quadratic fit (arcsec)")
plt.xlabel("Time (seconds)")

plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/TMA_DemandPosition_{expId}.png")
"""

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(5,5))
mtPos = getEfdData(client, 'lsst.sal.MTMount.azimuth',
                 expRecord=expRecord)

azVals = mtPos['demandPosition'].values
azTimes = mtPos['demandPositionTimestamp'].values
azTimes -= azTimes[0]
nPoints=len(mtPos)
counter = np.linspace(0, nPoints-1, nPoints)
azValFit = np.polyfit(counter, azVals, 1)
azFitVals = np.polyval(azValFit, counter)
azValErr = (azVals - azFitVals)
azTimeFit = np.polyfit(counter, azTimes, 1)
azFitTimes = np.polyval(azTimeFit, counter)
azTimeErr = (azTimes - azFitTimes)

ax.plot(counter, azTimeErr, label="Times", color='red')
ax1 = ax.twinx()
ax1.plot(counter, azValErr, label="Values", color='blue')
ax.legend(loc='upper right')
ax1.legend(loc='upper left')
plt.xlim(100,110)

In [ ]:
plt.plot(azTimes[100:110], azVals[100:110], marker='x')
plt.plot(azTimes[100:110], fitVals[100:110], marker='o')

In [ ]:
azErr[100:110]

In [ ]:
def calcDeltaT(params, args):
    # This calculates the deltaT needed
    # to make the median(error) = 0
    [values, valTimes, demand, demTimes] = args
    [deltaT] = params
    demandInterp = np.interp(valTimes, demTimes + deltaT, demand)
    error = (values - demandInterp) * 3600
    value = abs(np.median(error))
    return value


In [ ]:
fig = plt.figure(figsize = (10,5))
plt.subplots_adjust(wspace=0.5)
plt.suptitle(f"TMA tracking errors {expId}", fontsize=18)
azValues = np.asarray(mountData.azimuthData["actualPosition"])
azValTimes = np.asarray(mountData.azimuthData["actualPositionTimestamp"])
azValTimes -= azValTimes[0]
azDemand = np.asarray(mountData.azimuthData["demandPosition"])
azDemTimes = np.asarray(mountData.azimuthData["demandPositionTimestamp"])
azDemTimes -= azDemTimes[0]
elValues = np.asarray(mountData.elevationData["actualPosition"])
elValTimes = np.asarray(mountData.elevationData["actualPositionTimestamp"])
elValTimes -= elValTimes[0]
elDemand = np.asarray(mountData.elevationData["demandPosition"])
elDemTimes = np.asarray(mountData.elevationData["demandPositionTimestamp"])
elDemTimes -= elDemTimes[0]

azDemandInterp = np.interp(azValTimes, azDemTimes, azDemand)
elDemandInterp = np.interp(elValTimes, elDemTimes, elDemand)

azError = (azValues - azDemandInterp) * 3600
elError = (elValues - elDemandInterp) * 3600
plt.subplot(1,2,1)
plt.plot(azValTimes, azError)
plt.ylim(-0.1, 0.1)
plt.title("Azimuth")
plt.ylabel("actualPosition - demandPosition (arcsec)")
plt.xlabel("Time (seconds)")
plt.subplot(1,2,2)
plt.plot(elValTimes, elError)
plt.ylim(-0.1, 0.1)
plt.title("Elevation")
plt.ylabel("actualPosition - demandPosition (arcsec)")
plt.xlabel("Time (seconds)")

plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/TMA_actual-demand_{expId}.png")

In [ ]:
fig = plt.figure(figsize = (10,5))
plt.subplots_adjust(wspace=0.5)
plt.suptitle(f"TMA tracking errors {expId}", fontsize=18)
azPos = np.asarray(mountData.azimuthData["actualPosition"])
azPosTimes = np.asarray(mountData.azimuthData["actualPositionTimestamp"])
azPosTimes -= azPosTimes[0]
azDem = np.asarray(mountData.azimuthData["demandPosition"])
azDemTimes = np.asarray(mountData.azimuthData["demandPositionTimestamp"])
azDemTimes -= azDemTimes[0]
azPosFit = np.polyfit(azPosTimes, azPos, 1)
azPosFitVals = np.polyval(azPosFit, azPosTimes)
azPosErr = (azPos - azPosFitVals) * 3600
azDemFit = np.polyfit(azDemTimes, azDem, 1)
azDemFitVals = np.polyval(azDemFit, azDemTimes)
azDemErr = (azDem - azDemFitVals) * 3600

elPos = np.asarray(mountData.elevationData["actualPosition"])
elPosTimes = np.asarray(mountData.elevationData["actualPositionTimestamp"])
elPosTimes -= elPosTimes[0]
elDem = np.asarray(mountData.elevationData["demandPosition"])
elDemTimes = np.asarray(mountData.elevationData["demandPositionTimestamp"])
elDemTimes -= elDemTimes[0]
elPosFit = np.polyfit(elPosTimes, elPos, 1)
elPosFitVals = np.polyval(elPosFit, elPosTimes)
elPosErr = (elPos - elPosFitVals) * 3600
elDemFit = np.polyfit(elDemTimes, elDem, 1)
elDemFitVals = np.polyval(elDemFit, elDemTimes)
elDemErr = (elDem - elDemFitVals) * 3600
plt.subplot(1,2,1)
plt.plot(azPosTimes, azPosErr, marker='x', label="Position Error")
plt.plot(azDemTimes, azDemErr, marker='+', label="Demand Error")
plt.xlim(7.7, 8.5)
plt.legend()
plt.title("Azimuth")
plt.ylabel("deviation from linear fit (arcsec)")
plt.xlabel("Time (seconds)")
plt.subplot(1,2,2)
plt.plot(elPosTimes, elPosErr, marker='x', label="Position Error")
plt.plot(elDemTimes, elDemErr, marker='+', label="Demand Error")
plt.xlim(7.7, 8.5)
plt.legend()
plt.title("Elevation")
plt.ylabel("deviation from linear fit (arcsec)")
plt.xlabel("Time (seconds)")

plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/TMA_Position_Demand_Blowup_{expId}.png")

In [ ]:
plt.figure(figsize=(10,5))
plt.suptitle(f"Gill anemometer wind speed {expId}", fontsize=12)
anem= getEfdData(client, 'lsst.sal.ESS.airTurbulence', expRecord=expRecord)
speedVals = anem['speedMagnitude'].values
times = anem['timestamp']
times -= times[0]
plt.plot(times, -speedVals, marker='x')
plt.ylabel("Negative of SpeedMagnitude (m/s)")
plt.xlabel("Times(seconds)")
plt.xticks([0,5,10,15,20,25,30])
plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/Wind_Speed_{expId}.png")

In [ ]:
expId = 2025090200333
dataId = {'exposure':expId, 'instrument':'LSSTCam'}
expRecord = getExpRecordFromDataId(butler, dataId)

In [ ]:
plt.figure(figsize=(10,5))
plt.suptitle(f"Gill anemometer wind speed {expId}", fontsize=12)
anem= getEfdData(client, 'lsst.sal.ESS.airTurbulence', expRecord=expRecord)
speedVals = anem['speedMagnitude'].values
times = anem['timestamp']
times -= times[0]
plt.plot(times, -speedVals, marker='x')
plt.ylabel("Negative of SpeedMagnitude (m/s)")
plt.xlabel("Times(seconds)")
plt.xticks([0,5,10,15,20,25,30])
plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/Wind_Speed_{expId}.png")